In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio

from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

from lazypredict.Supervised import LazyClassifier, LazyRegressor

In [2]:
df = pd.read_csv("./Soil Nutrients.csv")
df.head()

,Name,Fertility,Photoperiod,Temperature,Rainfall,pH,Light_Hours,Light_Intensity,Rh,Nitrogen,Phosphorus,Potassium,Yield,Category_pH,Soil_Type,Season,N_Ratio,P_Ratio,K_Ratio
0,Strawberry,Moderate,Day Neutral,20.89,747.86,6.57,13.09,533.76,91.20,170.80,118.67,243.33,20.37,low_acidic,Loam,Summer,10.00,10.00,10.00
1,Strawberry,Moderate,Day Neutral,18.06,711.10,6.25,13.06,505.79,91.94,179.29,121.02,246.91,20.40,low_acidic,Loam,Spring,10.00,10.00,10.00
2,Strawberry,Moderate,Short Day Period,16.78,774.04,6.35,12.95,512.99,91.39,181.44,116.94,242.70,19.16,low_acidic,Loam,Summer,10.00,10.00,10.00
3,Strawberry,Moderate,Short Day Period,14.28,665.63,6.26,13.32,484.86,91.25,176.17,122.23,237.10,20.27,low_acidic,Loam,Summer,10.00,10.00,10.00
4,Strawberry,Moderate,Day Neutral,21.44,806.53,6.38,13.31,512.75,92.35,182.94,126.09,243.88,20.40,low_acidic,Loam,Spring,10.00,10.00,10.00


In [3]:
df = df.drop(columns=["Category_pH"])

In [4]:
encoder = LabelEncoder()
label_mappings = {}

for column in df.select_dtypes("object"):
    df[column] = encoder.fit_transform(df[column])
    label_mappings[column] = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))

In [5]:
pca = PCA(n_components=3)
pca_soil_nutrients = pca.fit_transform(df.drop(columns=["pH"]))
pca_soil_nutrients = pd.DataFrame(
    data=pca_soil_nutrients, columns=["PCA Component 1", "PCA Component 2", "PCA Component 3"]
)
pca_soil_nutrients

,PCA Component 1,PCA Component 2,PCA Component 3
0,-225.06,111.95,2.59
1,-257.68,82.00,18.38
2,-196.43,98.36,15.55
3,-298.20,51.11,15.54
4,-165.35,105.32,21.77
...,...,...,...
15395,149.93,-75.23,-19.28
15396,26.97,-71.11,-43.27
15397,20.91,-96.60,-34.71
15398,-58.44,-130.25,-33.12


In [ ]:
pca_soil_nutrients["pH"] = df["pH"]

In [7]:
class YieldRegressor:
    def __init__(self, data, features_for_regression, test_size=0.2, random_state=42):
        self.data = data
        self.features_for_regression = features_for_regression
        self.test_size = test_size
        self.random_state = random_state

        self.features = self.data.drop(self.features_for_regression, axis=1)
        self.targets = self.data[self.features_for_regression]
        self.features_train, self.features_test, self.targets_train, self.targets_test = train_test_split(
            self.features, self.targets, test_size=self.test_size, random_state=self.random_state
        )

        self.lazyRegressor = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None)
        self.models = None
        self.predictions = None

    def fit(self):
        self.models, self.predictions = self.lazyRegressor.fit(
            self.features_train, self.features_test, self.targets_train, self.targets_test
        )

    def get_models(self):
        return self.models

    def get_predictions(self):
        return self.predictions

In [8]:
yieldRegressor = YieldRegressor(pca_soil_nutrients, ["pH"])
yieldRegressor.fit()

 98%|█████████▊| 41/42 [00:44<00:00,  1.30it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000281 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 12320, number of used features: 3
[LightGBM] [Info] Start training from score 6.474166


100%|██████████| 42/42 [00:44<00:00,  1.07s/it]


In [ ]:
models = yieldRegressor.get_models().style.format(precision=5)
models

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
LGBMRegressor,0.72578,0.72605,0.23787,0.34769
HistGradientBoostingRegressor,0.72094,0.72121,0.23996,0.52272
XGBRegressor,0.71283,0.71311,0.24342,0.30853
RandomForestRegressor,0.71064,0.71092,0.24435,3.60635
KNeighborsRegressor,0.70821,0.70850,0.24537,0.03371
ExtraTreesRegressor,0.70528,0.70557,0.24660,1.23037
MLPRegressor,0.68959,0.68989,0.25308,3.29137
BaggingRegressor,0.68626,0.68657,0.25443,0.36021
GradientBoostingRegressor,0.64501,0.64536,0.27064,1.10565
